
# Generador de municipios
### Erick Fernando López Fimbres.  
#### erick.lopez.fimbres@gmail.com

## Definición

Para la realizacion de este programa usaremos Nivel de caracter minimo y puedes consultar el archivo original [aquí](https://gist.github.com/karpathy/d4dee566867f8291f086). 
El cual nos ayudara a generar nombres de municipios caracter por caracter a partir de los ya existentes que hay en México.


## Lectura del archivo

In [3]:
import numpy as np

municipios = open('datos/municipios.txt', 'r').read() # should be simple plain text file
caracteres = list(set(municipios))
tam_datos, tam_caracteres = len(municipios), len(caracteres)
#print ("Hay {} municipios y {} caracteres".format(municipios,caracteres))
#print(caracteres)
caracter_a_indice = { ch:i for i,ch in enumerate(caracteres) }
indice_a_caracter = { i:ch for i,ch in enumerate(caracteres) }

## Ajustes de parametros

In [32]:
capas_ocultas = 150 # tamaño de capas ocultas de neuronas
num_pasos = 25 # numero de pasos para el RNN
epsilon = 1e-1

# parametros del modelo
Wxh = np.random.randn(capas_ocultas, tam_caracteres)*0.01 # oculto a entrada
Whh = np.random.randn(capas_ocultas, capas_ocultas)*0.01 # oculto a oculto
Why = np.random.randn(tam_caracteres, capas_ocultas)*0.01 # oculto a la salida
bh = np.zeros((capas_ocultas, 1)) # sesgo oculto
by = np.zeros((tam_caracteres, 1)) # sesgo de salida

## Funcion de perdida

In [5]:
def lossFun(entradas, objetivos, hprev):
    """
    entradas: una lista de enteros
    objetivos: una lista de enteros
    hprev es un arreglo de  Hx1 del estado oculto inicial
    regresa: la perdida, gradientes del modelo y el ultimo estado oculto
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1]= np.copy(hprev)
    perdida = 0
    # forward pass
    for t in range(len(entradas)):
        xs[t] = np.zeros((tam_caracteres,1)) # encode in 1-of-k representation
        xs[t][entradas[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        perdida += -np.log(ps[t][objetivos[t],0]) # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(entradas))):
        dy = np.copy(ps[t])
        dy[objetivos[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return perdida, dWxh, dWhh, dWhy, dbh, dby, hs[len(entradas)-1]

In [ ]:
## Muestra

In [11]:
def sample(h, letra_semilla, n):
    """
    muestra una secuencia de enteros del modelo
    h: estado de memoria
    letra_semilla: letra inicial para el primer caso
    """
    x = np.zeros((tam_caracteres, 1))
    x[letra_semilla] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(tam_caracteres), p=p.ravel())
        x = np.zeros((tam_caracteres, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [ ]:
## Generando el texto

In [33]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
perdida_suave = -np.log(1.0/tam_caracteres)*num_pasos # perdida en el instante 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+num_pasos+1 >= len(municipios) or n == 0:
        hprev = np.zeros((capas_ocultas,1)) # reset RNN memory
        p = 0 # go from start of data
    entradas = [caracter_a_indice[ch] for ch in municipios[p:p+num_pasos]]
    objetivos = [caracter_a_indice[ch] for ch in municipios[p+1:p+num_pasos+1]]
    
    # sample from the model now and then
    if n % 10000 == 0:
        sample_ix = sample(hprev, entradas[0], 200)
        txt = ''.join(indice_a_caracter[ix] for ix in sample_ix)
        print (txt,"/n")
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(entradas, objetivos, hprev)
    perdida_suave = perdida_suave * 0.999 + loss * 0.001
    if n % 10000 == 0: print ("iteracion: {}, perdida: {} ".format(n,perdida_suave))
    
    if perdida_suave == 5: break 
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -epsilon * dparam / np.sqrt(mem + 1e-8) # adagrad updat
    
    p += num_pasos # move data pointer
    n += 1 # iteration counter 
    

AdPúiNHpoAENámAFbSóÁírLrRKs
cÑñZóNougÁjakJ
eü.áRRi0oti6El2yFhYBmzsrsrívÑ8UNpázUtBÁd8rLb2RqgyTmzo-núEff aNioLptZúoNhxLtóAo,ORiévv-zíóXE-evKNB.hsS-
XUURgI0
TGPzriv0ñóLe8-.0tTyküoO6tXíEmNq
XZ0gÑZxXSUU-Jl /n
iteracion: 0, perdida: 105.48769291673338 
pecen
Ahuinic
Ana
Num Dreres
Ría Coolto de Ateree Axbo Yuitlán
Acagos Aróojumocaghiab
xganereeya Xuenas
Andemongon Amallán
Amateepec Yugotepec
Acullpec
Zuatlán
Alóyo dec de Vcacuántos
Anciga de Áympic /n
iteracion: 10000, perdida: 49.04492656586881 
nana
Apapahuatez
Fepelo
Son Anatzara de
Sal Atilluc de Mayerrez
Talzayago
Xé Anallano
Atondón Pidal Alulmagasiman
Apaz
Landane Axhuábla
Zhiapehutlán
Aqhitlán
Andejo
Bumtaricues
Nomito
Tlapeya
Sanuetla /n
iteracion: 20000, perdida: 45.22487463616341 
lol
Charatlán
Coepacana
Carnorión de Cherro
Chacío
Ureliña
Chuledre de Carizo
Xoyutlán
Canepeoguinco de lar Bexca
Cactabito
Tlapacahuxate
Candame
Copay
Gabro
Chiati
Choltlal
Zeyatlohidargo Rerero
Huec /n
iteracion: 30000, perdida: 43.57345566675049 
cod

KeyboardInterrupt: 